# Building Electricity Consumption Assesment

#### Brief overview:

The energy demand for each cell is assessed according to the following parameters:
𝐵 Number of buildings
𝑆𝑟𝑒𝑠 Share of res buildings
𝑁 Nb of HH per res buildings
𝑎 Electrified status (probability)
𝐸_𝐻𝐻  Energy consumption per HH
𝑟 Adjustment with RWI

For each cell c, we have 𝐷_𝑐=𝐵_𝑐∗𝑆𝑟𝑒𝑠∗𝑁_𝑐  ∗𝑎_𝑐  ∗𝐸_𝐻𝐻  ∗𝑟_𝑐 

### Import necessary modules

In [1]:
# Check if we are running the notebook directly, if so move workspace to parent dir
import sys
import os
currentdir = os.path.abspath(os.getcwd())
if os.path.basename(currentdir) != 'EDeMOS':  
  sys.path.insert(0, os.path.dirname(currentdir))
  os.chdir('..')
  print(f'Move to {os.getcwd()}')

Move to C:\Users\amillot\PycharmProjects\EDeMOS


In [2]:
### Activate geospatial_env first

# Numeric
import numpy as np
import pandas as pd
import math
from scipy.optimize import fsolve

# Spatial
import geopandas as gpd
import fiona
from pyproj import CRS

# Mapping / Plotting
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from matplotlib.colors import LogNorm
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.colors as colors

import importlib
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
import config
importlib.reload(config)

INFO: Successfully loaded configuration for Zambia from config.py.
INFO: Successfully loaded configuration for Zambia from config.py.


<module 'config' from 'C:\\Users\\amillot\\PycharmProjects\\EDeMOS\\config.py'>

In [5]:
import plotting

In [6]:
from utils import processing_raster, finalizing_rasters, convert_features_to_geodataframe

In [7]:
import Buildings.data_loader
importlib.reload(Buildings.data_loader)
from Buildings.data_loader import load_initial_data, extract_raster_data, load_un_stats, load_census_data

In [8]:
import Buildings.residential_hh_analysis
importlib.reload(Buildings.residential_hh_analysis)
from Buildings.residential_hh_analysis import determine_location_status, determine_electrification_status, calculate_household_numbers, estimate_hh_with_access
import Buildings.residential_analysis
importlib.reload(Buildings.residential_analysis)
from Buildings.residential_analysis import calculate_energy_per_hh_method1, calculate_total_residential_electricity

In [9]:
import Buildings.figures_input_data
importlib.reload(Buildings.figures_input_data)
from Buildings.figures_input_data import *
from plotting import plot_sector_consumption_map

In [10]:
try: extracting_data
except NameError: extracting_data = True

## Import data

### Load initial data grid

In [11]:
# Load initial data (grid and administrative boundaries)
regions, admin_gdf, region_gdf, grid = load_initial_data(config)
print(grid.crs)

Loading initial data...
Admin boundaries loaded. Country GDF: (1, 3), Region GDF: (10, 12)
Hexagon grid loaded: (122901, 11)
Validating that all regions have cells in the pre-processed grid...
Validation successful: All target regions are present in the grid file.
EPSG:4326


### Extract raster values to hexagons

In [ ]:
# Extract raster data
if extracting_data:
    grid = extract_raster_data(grid, config, processing_raster, convert_features_to_geodataframe)
    grid.to_file(config.RESIDENTIAL_TEMP_FILE, layer="buildings", driver="GPKG", mode='w')
    print(grid.crs)

Extracting raster data...
EPSG:4326


In [ ]:
if not extracting_data:
    grid = gpd.read_file(config.RESIDENTIAL_TEMP_FILE)

### Extract residential and service demand from UN stats

In [ ]:
total_residential_elec_GWh, total_services_elec_GWh = load_un_stats(config)

### Load Census data

In [ ]:
data_HH, df_censusdata = load_census_data(config)

## Residential electricity consumption assessment

In [ ]:
plot_buildings_map(grid, config, fig_size=(10, 5))

In [ ]:
print(f"Total number of buildings: {grid[config.COL_BUILDINGS_SUM].sum():.0f}")

In [ ]:
plot_hrea_map(grid, config, fig_size=(10, 5))

### Step 1: assess the number of HH with access

#### Determine location (ruban or rural) of each cell

In [ ]:
grid = determine_location_status(grid, config)

In [ ]:
plot_urban_rural_map(grid, config, fig_size=(10, 5))

#### Determine electrifed status of each cell

In [ ]:
grid = determine_electrification_status(grid, config, admin_gdf)

In [ ]:
plot_line_proximity_map(grid, config, admin_gdf, fig_size=(10, 5))

#### Assess number of households per cell

In [ ]:
grid, df_HH_buildings = calculate_household_numbers(grid, config, data_HH, regions)

In [ ]:
df_HH_buildings

#### Assess number of households per cell with access to electricity

In [ ]:
grid, df_HH_buildings = estimate_hh_with_access(grid, config, df_HH_buildings, data_HH)

### Step 2: assess the electricity consumption per HH

#### Method 1: link the energy consumption to rwi through a logistic function

In [ ]:
# Normalise the rwi index
rwi_min = grid[config.COL_RWI_MEAN].min()
rwi_max = grid[config.COL_RWI_MEAN].max()
grid[config.COL_RWI_NORM] = (grid[config.COL_RWI_MEAN] - rwi_min) / (rwi_max - rwi_min)
grid[config.COL_RWI_NORM].plot.hist()
plt.show()

In [ ]:
# Plot of number of HH vs rwi

# Create equally spaced bins for the 'rwi' values
num_groups = 100
min_rwi = grid[config.COL_RWI_NORM].min()
max_rwi = grid[config.COL_RWI_NORM].max()
bin_width = (max_rwi - min_rwi) / num_groups
rwi_bins = [min_rwi + i * bin_width for i in range(num_groups + 1)]
rwi_bins_labels = [(rwi_bins[i] + rwi_bins[i])/2 for i in range(num_groups)]

# Group by the bins and sum the 'HH_total' values
grid['rwi_group'] = pd.cut(grid[config.COL_RWI_NORM], rwi_bins)
result = grid.groupby('rwi_group')[config.COL_HH_TOTAL].sum()
result.index = result.index.astype(str)
# # Print the result
# print(result)

# Create a bar plot
plt.figure(figsize=(10, 6))
plt.bar(result.index, result.values, color='skyblue', edgecolor='black')
# plt.bar(rwi_bins_labels, result.values, color='skyblue', edgecolor='black')
plt.xlabel('RWI Value Groups')
plt.ylabel('Total HH_total')
plt.title('Sum of HH_total by RWI Value Groups')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Plot of number of HH with access vs rwi

# Group by the bins and sum the 'HHwithAccess' values
result = grid.groupby('rwi_group')[config.COL_HH_WITH_ACCESS].sum()
result.index = result.index.astype(str)
# # Print the result
# print(result)

# Create a bar plot
plt.figure(figsize=(10, 6))
plt.rc('xtick', labelsize=5)
plt.bar(result.index, result.values, color='skyblue', edgecolor='black')
plt.xlabel('RWI Value Groups')
plt.ylabel('Total HH with access')
plt.title('Sum of HH with access by RWI Value Groups')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
grid, k_to_use = calculate_energy_per_hh_method1(grid, config, total_residential_elec_GWh)

In [ ]:
# create the curve linking energy consumption per HH and rwi
rwi_values = rwi_bins # rwi value groups
k = k_to_use  # Adjust this constant for the desired curve steepness
E_threshold = config.LOGISTIC_E_THRESHOLD
alpha = config.LOGISTIC_E_THRESHOLD / config.LOGISTIC_ALPHA_DERIVATION_THRESHOLD - 1
E_HH_values = config.LOGISTIC_E_THRESHOLD / (1 + alpha * np.exp(-k * np.array(rwi_values)))
# print(E_threshold / (1 + alpha * np.exp(-k  * 0)))
# Create the bar plot
plt.figure(figsize=(10, 6))
plt.bar(rwi_values, E_HH_values, color='skyblue', edgecolor='black')
plt.xlabel('RWI Value Groups')
plt.ylabel('Electricity consumption per household')
ax = plt.gca()  # Get the current axis
ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
# plt.title('Energy vs. RWI with logistic relationship')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# check that total energy assessed matches the statistics
grid[config.COL_RES_ELEC_PER_HH_LOG] = E_threshold / (1 + alpha * np.exp(-k * grid[config.COL_RWI_NORM]))

# Create the bar plot
plt.figure(figsize=(10, 6))
plt.scatter(grid[config.COL_RWI_NORM], grid[config.COL_RES_ELEC_PER_HH_LOG], color='skyblue', edgecolor='black')
plt.xlabel('normalised RWI')
plt.ylabel('Electricity consumption per household (kWh)')
ax = plt.gca()  # Get the current axis
ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
# plt.title('Energy vs. RWI with Logarithmic Relationship')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

#### Method 2: use data coming from the DHS survey

In [ ]:
import Buildings.HouseholdEnergyUse.estimate_energy_rwi_link_national_new
importlib.reload(Buildings.HouseholdEnergyUse.estimate_energy_rwi_link_national_new)
from Buildings.HouseholdEnergyUse.estimate_energy_rwi_link_national_new import estimate_energy_rwi_link_national

grid = estimate_energy_rwi_link_national(grid, config)   
col_elec_rural = 'elec_demand_kWh_rural' 
col_elec_urban = 'elec_demand_kWh_urban' 
    
if col_elec_rural not in grid.columns or col_elec_urban not in grid.columns:
    raise KeyError(f"Expected columns '{col_elec_rural}' or '{col_elec_urban}' not found after DHS estimation.")
grid[config.COL_RES_ELEC_PER_HH_DHS] = grid[col_elec_rural] + grid[col_elec_urban]

### Step 3: assess electricity consumption per cell

In [ ]:
grid = calculate_total_residential_electricity(grid, config, total_residential_elec_GWh)

### Compare access rates to Falchetta dataset

In [ ]:
def compare_access_to_falchetta(grid_gdf, app_config):
    """
    Compares calculated residential energy consumption tiers with Falchetta dataset tiers.

    This function bins calculated per-household energy into tiers and compares the 
    distribution of households across these tiers against pre-loaded Falchetta tier data.
    It also performs a similarity analysis between the DHS-based calculated tiers and 
    Falchetta's majority tier.

    Args:
        grid_gdf: GeoDataFrame of the hexagonal grid with energy consumption data.
        app_config: The configuration module.

    Returns:
        GeoDataFrame: grid_gdf, potentially with added columns for tiering/comparison.
    """
    print("Comparing access tiers to Falchetta dataset...")

    def calculate_tier_share_method(data_grid, method_suffix, hh_with_access_col, hh_wo_access_col, category_total_val):
        # Helper for tier share calculation
        tier_col_name = f'tiers_{method_suffix}' 
        if tier_col_name not in data_grid.columns:
            # print(f"Warning: Tier column '{tier_col_name}' not found for method '{method_suffix}'.")
            return pd.Series(dtype=float)
        if category_total_val == 0: return pd.Series(dtype=float)

        tier_share = data_grid.groupby(tier_col_name)[hh_with_access_col].sum()
        if 0 in tier_share.index :
            tier_share.loc[0] += data_grid[hh_wo_access_col].sum()
        else: 
            tier_share.loc[0] = data_grid[hh_wo_access_col].sum()
        return tier_share.sort_index() / category_total_val

    bins_tiers = app_config.BINS_TIERS_ENERGY
    tier_labels = range(len(bins_tiers) - 1)

    categories_summary = {
        'national': app_config.COL_HH_TOTAL, 'urban': app_config.COL_HH_URBAN, 'rural': app_config.COL_HH_RURAL
    }

    # Falchetta dataset
    for col_type in [app_config.COL_TIERS_FALCHETTA_MAJ, app_config.COL_TIERS_FALCHETTA_MEAN]:
        if col_type in grid_gdf.columns:
            tiers_summary_df = pd.DataFrame()
            for cat_name, total_hh_col in categories_summary.items():
                 if total_hh_col in grid_gdf.columns and grid_gdf[total_hh_col].sum() > 0:
                    cat_sum = grid_gdf.groupby(col_type)[total_hh_col].sum()
                    tiers_summary_df[cat_name] = cat_sum / cat_sum.sum()
            print(f"\nFalchetta Tiers Summary ({col_type}):")
            print(tiers_summary_df.fillna(0))
    
    # Our methods
    methods_to_compare = {
        'meth1': app_config.COL_RES_ELEC_PER_HH_LOG,
        'meth2': "ElecPerHH_kWh_meth2" 
    }
    categories_for_comparison = [
        ('national', app_config.COL_HH_WITH_ACCESS, app_config.COL_HH_WO_ACCESS, app_config.COL_HH_TOTAL),
        ('urban', app_config.COL_HH_WITH_ACCESS_URB, app_config.COL_HH_WO_ACCESS_URB, app_config.COL_HH_URBAN),
        ('rural', app_config.COL_HH_WITH_ACCESS_RUR, app_config.COL_HH_WO_ACCESS_RUR, app_config.COL_HH_RURAL)
    ]

    for method_key, energy_col_name in methods_to_compare.items():
        if energy_col_name not in grid_gdf.columns:
            print(f"Warning: Energy column '{energy_col_name}' for method '{method_key}' not found.")
            continue
        
        grid_gdf[f'tiers_{method_key}'] = pd.cut(grid_gdf[energy_col_name], bins=bins_tiers, labels=tier_labels, right=False)
        grid_gdf[f'tiers_{method_key}'] = grid_gdf[f'tiers_{method_key}'].fillna(0).astype(int)

        df_tiers_data = pd.DataFrame()
        for cat_name, hh_access_col, hh_no_access_col, total_hh_col in categories_for_comparison:
            if all(c in grid_gdf.columns for c in [hh_access_col, hh_no_access_col, total_hh_col]):
                cat_total_val = grid_gdf[total_hh_col].sum()
                if cat_total_val > 0:
                    tier_share_series = calculate_tier_share_method(grid_gdf, method_key, hh_access_col, hh_no_access_col, cat_total_val)
                    df_tiers_data[cat_name] = tier_share_series
        
        print(f"\nTier Shares for Method '{method_key}':")
        print(df_tiers_data.fillna(0))

    if f'tiers_meth2' in grid_gdf.columns and app_config.COL_TIERS_FALCHETTA_MAJ in grid_gdf.columns:
        grid_gdf['tiers_DHS_adjusted'] = grid_gdf['tiers_meth2'].where(grid_gdf['tiers_meth2'] != 5, 4) 
        grid_gdf['Similarity_Falchetta_DHS'] = grid_gdf['tiers_DHS_adjusted'] == grid_gdf[app_config.COL_TIERS_FALCHETTA_MAJ]
        grid_gdf['Difference_Falchetta_DHS'] = abs(pd.to_numeric(grid_gdf['tiers_DHS_adjusted']) - pd.to_numeric(grid_gdf[app_config.COL_TIERS_FALCHETTA_MAJ]))

        print("\nSimilarity Analysis (Falchetta vs DHS-Method2):")
        print(f"Number of lines with similar tiers: {grid_gdf['Similarity_Falchetta_DHS'].sum()}")
        print(f"Mean difference in tiers: {grid_gdf['Difference_Falchetta_DHS'].mean():.2f}")
        print(f"Median difference in tiers: {grid_gdf['Difference_Falchetta_DHS'].median():.2f}")
    
    print("Finished Falchetta comparison.")
    return grid_gdf

In [ ]:
grid = compare_access_to_falchetta(grid, config)

### Final grid

In [ ]:
grid[config.COL_RES_ELEC_KWH_FINAL]=grid[config.COL_RES_ELEC_KWH_METH2_SCALED]

In [ ]:
# print(grid.columns)
grid.to_csv(config.RESIDENTIAL_GRID_FILE)

In [ ]:
# Convert categorical columns to strings
for col in grid.select_dtypes(include=['category']).columns:
    #print(col)
    grid[col] = grid[col].astype(str)

In [ ]:
if 'rwi_group' in grid.columns:
    grid = grid.drop('rwi_group', axis=1)
if 'tiers_DHS' in grid.columns:
    grid = grid.drop('tiers_DHS', axis=1)
if 'bin_labels' in grid.columns:
    grid = grid.drop('bin_labels', axis=1)
grid.to_file(config.RESIDENTIAL_OUTPUT_DIR / f'res_energy_map_{config.COUNTRY}.shp', index=False)
grid.head(3)

### Map residential results

In [ ]:
plot_sector_consumption_map(grid, config.COL_RES_ELEC_KWH_METH2_SCALED, config, admin_gdf, region_gdf, 'Residential', fig_size=(25, 15))

# Services

In [ ]:
import Buildings.service_analysis
importlib.reload(Buildings.service_analysis)
from Buildings.service_analysis import calculate_service_buildings_based_elec, calculate_service_gdp_based_elec, calculate_employee_based_electricity

## Electricity consumption based on number of buildings with access

In [ ]:
grid = calculate_service_buildings_based_elec(grid, config, total_services_elec_GWh)

## Energy consumption based on GDP

In [ ]:
# grid = calculate_service_gdp_based_elec(grid, config, total_services_elec_GWh)

## Energy consumption based on employees

In [ ]:
grid = calculate_employee_based_electricity(grid, config, total_services_elec_GWh, df_censusdata)

## Weighted average of the three assessements

In [ ]:
# if config.COL_GDP_PPP_MEAN in grid.columns:
#     # Link between buildings and GDP
#     plt.scatter(grid['serBUi_Acc'], grid['GDP_PPP'],s=1)
    
#     # Add labels to the plot
#     plt.xlabel('serBUi_Acc')
#     plt.ylabel('GDP_PPP')
    
#     # Set the axis to logarithmic scale
#     # plt.yscale('log')
#     # plt.xscale('log')
    
#     # Show the plot
#     plt.show()

In [ ]:
threshold_access = 0.1 # lower value than residential because easier to connect services buildings
alpha = config.SERVICES_WEIGHT_GDP
beta = config.SERVICES_WEIGHT_BUILDINGS
gama = config.SERVICES_WEIGHT_EMPLOYEES

In [ ]:
# compute weighted average
grid['SElec_kWh_weighted'] = (beta * grid[config.COL_SER_ELEC_KWH_BUI] + gama * grid[config.COL_SER_ELEC_KWH_EMP] )
totalSEn_kWh_weighted = grid['SElec_kWh_weighted'].sum()
grid[config.COL_SER_ELEC_KWH_FINAL] = grid['SElec_kWh_weighted'] / totalSEn_kWh_weighted * total_services_elec_GWh *10**6 

## Results per region and map

In [ ]:
services_result = pd.DataFrame()
services_result = grid.groupby(config.COL_ADMIN_NAME)[config.COL_SER_ELEC_KWH_FINAL].sum() / 10**6 # conversion in GWh
pd.options.display.float_format = '{:.2f}'.format
services_result

In [ ]:
grid[config.COL_ADMIN_NAME].unique()

In [ ]:
plot_sector_consumption_map(grid, config.COL_SER_ELEC_KWH_FINAL, config, admin_gdf, region_gdf, 'Services', fig_size=(25, 15))

In [ ]:
grid.to_csv(config.SERVICES_GRID_FILE)
# grid.to_file(config.RESIDENTIAL_OUTPUT_DIR / f'ser_energy_map_{config.COUNTRY}.shp', index=False)
# grid.to_file(config.RESIDENTIAL_OUTPUT_DIR /  f'ser_energy_map_{config.COUNTRY}.geojson', driver='GeoJSON', index=False)
grid.to_file(config.BUILDINGS_GPKG_FILE, layer="buildings", driver="GPKG", mode='w')
grid.head(3)

In [ ]:
total_services_elec_scaled_kwh = grid.groupby(config.COL_ADMIN_NAME)[config.COL_SER_ELEC_KWH_FINAL].sum()
# Reindex the result using your master list of regions.
#    This ensures every region has an entry. Missing ones will be filled with NaN.
total_services_elec_scaled_kwh = total_services_elec_scaled_kwh.reindex(regions)
# Conversion to GWh.
total_services_elec_scaled_gwh = total_services_elec_scaled_kwh / 10**6
print("Services electricity consumption assessed after scaling:")

for region in regions:
    value_gwh = total_services_elec_scaled_gwh[region]    
    # Check if the value is NaN before trying to format it as a number
    if pd.isna(value_gwh):
        print(f"{region}: NaN GWh")
    else:
        print(f"{region}: {value_gwh:,.0f} GWh")

# Buildings

In [ ]:
# total_Buienergy_scaled = total_servicesenergy_scaled + total_residentialenergy_scaled
grid[config.COL_BUI_ELEC_KWH_FINAL]=grid[config.COL_RES_ELEC_KWH_FINAL]+grid[config.COL_SER_ELEC_KWH_FINAL]
total_buildings_elec_scaled = grid.groupby(config.COL_ADMIN_NAME)[config.COL_BUI_ELEC_KWH_FINAL].sum()
total_buildings_elec_scaled = total_buildings_elec_scaled.reindex(regions)
print ("Buildings electricity consumption assessed after scaling:")
for region in regions:
    print (region, f"{total_buildings_elec_scaled[region]:,.0f}", "GWh" )